In [1]:
import torch
import torch_geometric
import numpy as np
from torch_geometric.data import Data

### Read In Edge List and Transpose into COO Format

In [2]:
with open('edge_list/edge_index.txt') as edge_file:
    edges = []
    for line in edge_file.readlines():
        u, v, d = line.split()
        edges.append([int(u),int(v)])
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

### Read in Data Matrix and Convert to Torch Tensor

$n x m$ matrix where $n$ is the number of states and $m$ is the number of search terms(features).

In [9]:
!ls hopkins_data/cleaned

01-22-2020.csv 02-09-2020.csv 02-27-2020.csv 03-16-2020.csv 04-03-2020.csv
01-23-2020.csv 02-10-2020.csv 02-28-2020.csv 03-17-2020.csv 04-04-2020.csv
01-24-2020.csv 02-11-2020.csv 02-29-2020.csv 03-18-2020.csv 04-05-2020.csv
01-25-2020.csv 02-12-2020.csv 03-01-2020.csv 03-19-2020.csv 04-06-2020.csv
01-26-2020.csv 02-13-2020.csv 03-02-2020.csv 03-20-2020.csv 04-07-2020.csv
01-27-2020.csv 02-14-2020.csv 03-03-2020.csv 03-21-2020.csv 04-08-2020.csv
01-28-2020.csv 02-15-2020.csv 03-04-2020.csv 03-22-2020.csv 04-09-2020.csv
01-29-2020.csv 02-16-2020.csv 03-05-2020.csv 03-23-2020.csv 04-10-2020.csv
01-30-2020.csv 02-17-2020.csv 03-06-2020.csv 03-24-2020.csv 04-11-2020.csv
01-31-2020.csv 02-18-2020.csv 03-07-2020.csv 03-25-2020.csv 04-12-2020.csv
02-01-2020.csv 02-19-2020.csv 03-08-2020.csv 03-26-2020.csv 04-13-2020.csv
02-02-2020.csv 02-20-2020.csv 03-09-2020.csv 03-27-2020.csv 04-14-2020.csv
02-03-2020.csv 02-21-2020.csv 03-10-2020.csv 03-28-2020.csv 04-15-2020.csv
02-04-2020.csv 02-22-2020

In [4]:
nodes_props = np.loadtxt('./datesprepped/2020-01-31.csv')
nodes_props

array([[ 0.,  0.,  0., ...,  0., 11.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 14.,  0., ...,  0., 10.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

### Get Hopkins Data Labels

In [34]:
# y = np.round(np.random.random(size=[51,1]) * 10) # randomly generate some cases for now
# y = torch.tensor(y, dtype=torch.float)
with open('./hopkins_data/cleaned/2020-01-31.csv') as fy:
    y_arr = [int(line.split(',')[0]) for line in fy.readlines()]
y = torch.tensor(y_arr, dtype=torch.float).reshape([51, 1])

### Create Data Object

In [35]:
x = torch.tensor(nodes_props.tolist(), dtype=torch.float)
edge_index = edge_index
data = Data(x=x, y=y, edge_index=edge_index)

In [36]:
data.num_node_features

3243

### Creating Dataloader

Now that we've created a single data object as an example we need to create a torch dataloader. This way we can batch multiple nodes together.

In [47]:
import os
import os.path as osp
['x/' + f for f in os.listdir('raw_dir/x/')]

['x/2020-02-12.csv',
 'x/2020-02-07.csv',
 'x/2020-02-13.csv',
 'x/2020-02-11.csv',
 'x/2020-02-10.csv',
 'x/2020-02-28.csv',
 'x/2020-02-14.csv',
 'x/2020-02-15.csv',
 'x/2020-02-01.csv',
 'x/2020-02-17.csv',
 'x/2020-02-16.csv',
 'x/2020-03-19.csv',
 'x/2020-03-25.csv',
 'x/2020-03-31.csv',
 'x/2020-03-30.csv',
 'x/2020-03-24.csv',
 'x/2020-03-18.csv',
 'x/2020-04-07.csv',
 'x/2020-03-26.csv',
 'x/2020-03-27.csv',
 'x/2020-03-23.csv',
 'x/2020-03-22.csv',
 'x/2020-01-31.csv',
 'x/2020-03-20.csv',
 'x/2020-03-08.csv',
 'x/2020-03-09.csv',
 'x/2020-03-21.csv',
 'x/2020-03-04.csv',
 'x/2020-03-10.csv',
 'x/2020-03-11.csv',
 'x/2020-03-05.csv',
 'x/2020-03-13.csv',
 'x/2020-03-07.csv',
 'x/2020-03-06.csv',
 'x/2020-03-12.csv',
 'x/2020-03-16.csv',
 'x/2020-03-02.csv',
 'x/2020-03-03.csv',
 'x/2020-03-17.csv',
 'x/2020-03-01.csv',
 'x/2020-04-08.csv',
 'x/2020-03-15.csv',
 'x/2020-03-29.csv',
 'x/2020-03-28.csv',
 'x/2020-03-14.csv',
 'x/2020-02-27.csv',
 'x/2020-02-26.csv',
 'x/2020-02-2

In [129]:
import os.path as osp
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader


class COVIDSearchTerms(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(COVIDSearchTerms, self).__init__(root, transform, pre_transform)
        self.node_files = []
        self.target_files = []

    @property
    def raw_file_names(self):
        self.node_files = ['x/' + f for f in os.listdir('raw/x/')]
        # ensure that we only grab targets for dates we have
        self.target_files = [
            'y/' + f for f in
            list(set(os.listdir('raw/y/')) & set(os.listdir('raw/x/')))
        ]
        return self.node_files + self.target_files

    @property
    def processed_file_names(self):
        dates = os.listdir('raw/y/')
        return dates

    def download(self):
        pass
        # Download to `self.raw_dir`.

    def process(self):
        with open('edge_list/edge_index.txt') as edge_file:
            edges = []
            for line in edge_file.readlines():
                u, v, d = line.split()
                edges.append([int(u),int(v)])
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        
        i = 0
        for node_file in self.node_files:
            date = node_file.split('/')[-1].split('.')[0]
            x = torch.tensor(np.loadtxt('raw/' + node_file).tolist(), dtype=torch.float)
            with open('raw/y/' + date + '.csv') as fy:
                y_arr = [int(line.split(',')[0]) for line in fy.readlines()]
            y = torch.tensor(y_arr, dtype=torch.float).reshape([1, 51])
            edge_index = edge_index
            # Read data from `raw_path`.
            data = Data(x=x, y=y, edge_index=edge_index)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)
            torch.save(data, osp.join(self.processed_dir, 'data-{}.pt'.format(i)))
            i = i + 1

    def len(self):
        return 57

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, 'data-{}.pt'.format(idx)))
        return data

### Example Network Structure

In [130]:
'''
    Network structure taken from example:
    https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8
'''

feature_dim = data.num_node_features # should be 3243 for the number of queries
from torch_geometric.nn import TopKPooling, SAGEConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Replaced all instances of the dim 128 with 1500
        self.conv1 = SAGEConv(feature_dim, 1500)
        self.pool1 = TopKPooling(1500, ratio=0.8)
        self.conv2 = SAGEConv(1500, 1500)
        self.pool2 = TopKPooling(1500, ratio=0.8)
        self.conv3 = SAGEConv(1500, 1500)
        self.pool3 = TopKPooling(1500, ratio=0.8)
# We kind of did our own embeddings....
#         self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(3000, 1500)
#         self.lin1 = torch.nn.Linear(feature_dim, 1500)
        self.lin2 = torch.nn.Linear(1500, 750)
        self.lin3 = torch.nn.Linear(750, 51) # we probs need another layer
        self.bn1 = torch.nn.BatchNorm1d(1500)
        self.bn2 = torch.nn.BatchNorm1d(750)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()
        self.act3 = torch.nn.ReLU()
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
#         x, edge_index = data.x, data.edge_index
# Remove the embedding since we don't need it
#         x = self.item_embedding(x)
#         x = x.squeeze(1)        # Squeeze down x dim?

# We need to figure out some batches before we can pool?
#         print(self.conv1(x, edge_index))
    
        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
        # Using the third layer in a traditional sense?
        x = self.lin3(x)
        x = self.act3(x)
        # We need to change the ouput to continous rather than discrete
#         x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

### Example feed forward of the network

In [ ]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        print(output[:1],label[:1])
        print(loss)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(dataset)

batch_size=10
dataset = COVIDSearchTerms('.')
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.MSELoss()
train_loader = DataLoader(dataset, batch_size=batch_size)
for epoch in range(500):
    train()

### Old training example

In [38]:
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.MSELoss()


# Fit?
model.train()

for i in range(0,500):
    data = data.to(device)
    optimizer.zero_grad()
    output = model(data)
    label = data.y.to(device)
    loss = crit(output, label)
    print(loss)
    loss.backward()
    loss_all = loss.item()
    optimizer.step()

optimizer.zero_grad()
output = model(data)
loss = crit(output, label)
print(output)
print(loss)

tensor(0.6053, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBac

tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBac

tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBackward>)
tensor(0.1961, grad_fn=<MseLossBac

### Example work for earlier session with torch geo

In [2]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 2, 0, 3],
                           [1, 0, 1, 3, 2]], dtype=torch.long)

In [60]:
data = Data(x=x, edge_index=edge_index, y=y)

In [61]:
data.num_node_features

2

In [73]:
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(2, 1)
        self.lin1 = torch.nn.Linear(1, 1)
        self.act1 = torch.nn.ReLU()
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.relu(self.conv1(x, edge_index))
        x = self.lin1(x)
        x = self.act1(x)
        return x
        

In [91]:
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.MSELoss()


# Fit?
model.train()

for i in range(0,1000):
    data = data.to(device)
    optimizer.zero_grad()
    output = model(data)
    print(output, loss)
    label = data.y.to(device)
    loss = crit(output, label)
    loss.backward()
    loss_all = loss.item()
    optimizer.step()

optimizer.zero_grad()
output = model(data)
loss = crit(output, label)
print(output)
print(loss)

tensor([[0.6239],
        [1.3403],
        [1.2010],
        [0.7558]], grad_fn=<ReluBackward0>) tensor(0.2530, grad_fn=<MseLossBackward>)
tensor([[0.5991],
        [1.2942],
        [1.1583],
        [0.7262]], grad_fn=<ReluBackward0>) tensor(0.5696, grad_fn=<MseLossBackward>)
tensor([[0.5746],
        [1.2488],
        [1.1163],
        [0.6971]], grad_fn=<ReluBackward0>) tensor(0.5312, grad_fn=<MseLossBackward>)
tensor([[0.5506],
        [1.2043],
        [1.0752],
        [0.6685]], grad_fn=<ReluBackward0>) tensor(0.4962, grad_fn=<MseLossBackward>)
tensor([[0.5271],
        [1.1608],
        [1.0349],
        [0.6406]], grad_fn=<ReluBackward0>) tensor(0.4645, grad_fn=<MseLossBackward>)
tensor([[0.5042],
        [1.1183],
        [0.9957],
        [0.6134]], grad_fn=<ReluBackward0>) tensor(0.4358, grad_fn=<MseLossBackward>)
tensor([[0.4818],
        [1.0770],
        [0.9576],
        [0.5870]], grad_fn=<ReluBackward0>) tensor(0.4102, grad_fn=<MseLossBackward>)
tensor([[0.4601],
  

tensor([[0.2908],
        [0.6438],
        [0.5972],
        [0.3822]], grad_fn=<ReluBackward0>) tensor(0.2721, grad_fn=<MseLossBackward>)
tensor([[0.2904],
        [0.6417],
        [0.5958],
        [0.3821]], grad_fn=<ReluBackward0>) tensor(0.2719, grad_fn=<MseLossBackward>)
tensor([[0.2900],
        [0.6395],
        [0.5945],
        [0.3819]], grad_fn=<ReluBackward0>) tensor(0.2718, grad_fn=<MseLossBackward>)
tensor([[0.2897],
        [0.6374],
        [0.5931],
        [0.3818]], grad_fn=<ReluBackward0>) tensor(0.2716, grad_fn=<MseLossBackward>)
tensor([[0.2893],
        [0.6353],
        [0.5918],
        [0.3816]], grad_fn=<ReluBackward0>) tensor(0.2714, grad_fn=<MseLossBackward>)
tensor([[0.2890],
        [0.6332],
        [0.5905],
        [0.3815]], grad_fn=<ReluBackward0>) tensor(0.2713, grad_fn=<MseLossBackward>)
tensor([[0.2887],
        [0.6312],
        [0.5893],
        [0.3814]], grad_fn=<ReluBackward0>) tensor(0.2711, grad_fn=<MseLossBackward>)
tensor([[0.2885],
  

        [0.4220]], grad_fn=<ReluBackward0>) tensor(0.2636, grad_fn=<MseLossBackward>)
tensor([[0.3173],
        [0.5882],
        [0.5816],
        [0.4226]], grad_fn=<ReluBackward0>) tensor(0.2635, grad_fn=<MseLossBackward>)
tensor([[0.3178],
        [0.5877],
        [0.5815],
        [0.4231]], grad_fn=<ReluBackward0>) tensor(0.2635, grad_fn=<MseLossBackward>)
tensor([[0.3183],
        [0.5872],
        [0.5814],
        [0.4237]], grad_fn=<ReluBackward0>) tensor(0.2634, grad_fn=<MseLossBackward>)
tensor([[0.3188],
        [0.5867],
        [0.5813],
        [0.4242]], grad_fn=<ReluBackward0>) tensor(0.2633, grad_fn=<MseLossBackward>)
tensor([[0.3193],
        [0.5862],
        [0.5812],
        [0.4248]], grad_fn=<ReluBackward0>) tensor(0.2632, grad_fn=<MseLossBackward>)
tensor([[0.3198],
        [0.5858],
        [0.5811],
        [0.4253]], grad_fn=<ReluBackward0>) tensor(0.2631, grad_fn=<MseLossBackward>)
tensor([[0.3203],
        [0.5853],
        [0.5810],
        [0.4258]], g

tensor([[0.3578],
        [0.5554],
        [0.5690],
        [0.4568]], grad_fn=<ReluBackward0>) tensor(0.2575, grad_fn=<MseLossBackward>)
tensor([[0.3583],
        [0.5551],
        [0.5688],
        [0.4571]], grad_fn=<ReluBackward0>) tensor(0.2575, grad_fn=<MseLossBackward>)
tensor([[0.3587],
        [0.5548],
        [0.5687],
        [0.4573]], grad_fn=<ReluBackward0>) tensor(0.2574, grad_fn=<MseLossBackward>)
tensor([[0.3591],
        [0.5545],
        [0.5685],
        [0.4576]], grad_fn=<ReluBackward0>) tensor(0.2574, grad_fn=<MseLossBackward>)
tensor([[0.3596],
        [0.5543],
        [0.5683],
        [0.4579]], grad_fn=<ReluBackward0>) tensor(0.2573, grad_fn=<MseLossBackward>)
tensor([[0.3600],
        [0.5540],
        [0.5682],
        [0.4582]], grad_fn=<ReluBackward0>) tensor(0.2573, grad_fn=<MseLossBackward>)
tensor([[0.3605],
        [0.5537],
        [0.5680],
        [0.4584]], grad_fn=<ReluBackward0>) tensor(0.2572, grad_fn=<MseLossBackward>)
tensor([[0.3609],
  

tensor([[0.3899],
        [0.5380],
        [0.5559],
        [0.4732]], grad_fn=<ReluBackward0>) tensor(0.2544, grad_fn=<MseLossBackward>)
tensor([[0.3902],
        [0.5378],
        [0.5557],
        [0.4734]], grad_fn=<ReluBackward0>) tensor(0.2544, grad_fn=<MseLossBackward>)
tensor([[0.3906],
        [0.5377],
        [0.5555],
        [0.4736]], grad_fn=<ReluBackward0>) tensor(0.2543, grad_fn=<MseLossBackward>)
tensor([[0.3910],
        [0.5375],
        [0.5554],
        [0.4737]], grad_fn=<ReluBackward0>) tensor(0.2543, grad_fn=<MseLossBackward>)
tensor([[0.3913],
        [0.5373],
        [0.5552],
        [0.4739]], grad_fn=<ReluBackward0>) tensor(0.2543, grad_fn=<MseLossBackward>)
tensor([[0.3917],
        [0.5372],
        [0.5551],
        [0.4740]], grad_fn=<ReluBackward0>) tensor(0.2542, grad_fn=<MseLossBackward>)
tensor([[0.3921],
        [0.5370],
        [0.5549],
        [0.4742]], grad_fn=<ReluBackward0>) tensor(0.2542, grad_fn=<MseLossBackward>)
tensor([[0.3924],
  

...

tensor([[0.4719],
        [0.5085],
        [0.5156],
        [0.4957]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4720],
        [0.5085],
        [0.5155],
        [0.4957]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4721],
        [0.5085],
        [0.5155],
        [0.4957]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4723],
        [0.5084],
        [0.5154],
        [0.4958]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4724],
        [0.5084],
        [0.5153],
        [0.4958]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4725],
        [0.5084],
        [0.5153],
        [0.4958]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4726],
        [0.5083],
        [0.5152],
        [0.4958]], grad_fn=<ReluBackward0>) tensor(0.2503, grad_fn=<MseLossBackward>)
tensor([[0.4727],
  

        [0.4969]], grad_fn=<ReluBackward0>) tensor(0.2502, grad_fn=<MseLossBackward>)
tensor([[0.4794],
        [0.5062],
        [0.5115],
        [0.4969]], grad_fn=<ReluBackward0>) tensor(0.2502, grad_fn=<MseLossBackward>)
tensor([[0.4795],
        [0.5062],
        [0.5114],
        [0.4969]], grad_fn=<ReluBackward0>) tensor(0.2502, grad_fn=<MseLossBackward>)
tensor([[0.4796],
        [0.5062],
        [0.5114],
        [0.4970]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4797],
        [0.5062],
        [0.5113],
        [0.4970]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4798],
        [0.5061],
        [0.5113],
        [0.4970]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4799],
        [0.5061],
        [0.5112],
        [0.4970]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4799],
        [0.5061],
        [0.5112],
        [0.4970]], g

tensor([[0.4850],
        [0.5045],
        [0.5084],
        [0.4978]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4851],
        [0.5045],
        [0.5084],
        [0.4978]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4851],
        [0.5045],
        [0.5083],
        [0.4978]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4852],
        [0.5045],
        [0.5083],
        [0.4978]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4853],
        [0.5045],
        [0.5083],
        [0.4978]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4853],
        [0.5044],
        [0.5082],
        [0.4978]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4854],
        [0.5044],
        [0.5082],
        [0.4979]], grad_fn=<ReluBackward0>) tensor(0.2501, grad_fn=<MseLossBackward>)
tensor([[0.4854],
  

tensor([[0.4886],
        [0.5035],
        [0.5064],
        [0.4983]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4887],
        [0.5034],
        [0.5064],
        [0.4983]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4887],
        [0.5034],
        [0.5063],
        [0.4984]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4888],
        [0.5034],
        [0.5063],
        [0.4984]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4888],
        [0.5034],
        [0.5063],
        [0.4984]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4889],
        [0.5034],
        [0.5062],
        [0.4984]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4889],
        [0.5034],
        [0.5062],
        [0.4984]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4890],
  

tensor([[0.4914],
        [0.5026],
        [0.5048],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4915],
        [0.5026],
        [0.5048],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4915],
        [0.5026],
        [0.5048],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4915],
        [0.5026],
        [0.5047],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4916],
        [0.5026],
        [0.5047],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4916],
        [0.5025],
        [0.5047],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4917],
        [0.5025],
        [0.5047],
        [0.4988]], grad_fn=<ReluBackward0>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([[0.4917],
  